# Fine Tuning Large Language Model - Model

In this workshop, you will learn how to fine tune the prompts and the LLMs to enhance and improves its response.

In [52]:
# Import libraries
import torch, time
import pandas as pd
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, GenerationConfig, TrainingArguments

from peft import PeftModel, LoraConfig, get_peft_model, TaskType

In [ ]:
# Load and explore the following datasets
# Q: Number of sets? 
# Q: How many records in each of these sets?
# Q: What are the column names?

dataset_name = "knkarthick/dialogsum"
model_name = "google/flan-t5-small"
model_name = "google/flan-t5-base"

dataset = load_dataset(dataset_name)

print(dataset)
print(dataset.shape)

In [ ]:
# Print a record
idx = 100 

for k, v in dataset['train'][idx].items():
   print(f'{k.upper()}\n{v}\n')

## Fine tuning the LLM model

In this workshop we will be turning the <code>google/flan-t5-base</code> model.

In [55]:
# Utility function to dump a model's tunable parameters

def print_trainable_model_params(model):
   trainable_model_params = 0
   all_model_params = 0
   for _, param in model.named_parameters():
      all_model_params += param.numel()
      if param.requires_grad:
         trainable_model_params += param.numel()
   return f"Trainable parameters: {trainable_model_params}\nTotal parameters: {all_model_params}\nPercentable of trainable parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [56]:
# TODO: Load model



In [ ]:
# TODO: Print number of trainable parameters


### Preprocess the dialogue dataset

We will train the model to summarize dialogue by creating a dialogue-summary pair for the LLM to process. The dialogue is the training data and the summary is the label. This is supervized learning.

The prompt will be as follows

```
Summarize the following dialogue.\n
\n
Fred: ...\n
Barney: ...\n
\n
Summary:\n
Summary of the conversation between Fred and Barney
```

The prompt and the summary will be tokenized for the LLM

In [58]:
# Utitlity function to prepare the data for training 
# Tokenize function
# Need to create a tokenizer before calling this
def tokenize_fn(data):
   start_prompt = 'Summarize the following dialogue.\n\n'
   end_prompt = '\n\nSummary:'
   prompt = [ start_prompt + d + end_prompt for d in data['dialogue'] ]
   summary = data['summary']
   data['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
   data['labels'] = tokenizer(summary, padding="max_length", truncation=True, return_tensors="pt").input_ids
   return data


In [ ]:
# TODO: prepare the data for training with the tokenize_fn function
# Tokenize the 3 splits of the dataset: train, validation, test



In [ ]:
# TODO: Verify prepared data



In [61]:
# TODO: Remove id, dialogue, summary and topic columns from dataset. We only want input_ids and labels



In [ ]:
# TODO: Verify dataset again



### Tune model with pre-processed dataset

We will use [<code>Trainer</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#api-reference%20][%20transformers.Trainer) to train the original model. The training result will be written out. The trainer will be configure with [<code>TrainingArgument</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)

In [ ]:
# CUDA information
# We have install the torch CPU version
# pip3 install torch==2.5.1+cpu --index-url https://download.pytorch.org/whl/cpu
# To install CUDA version
# pip3 install torch

print('CUDA available: ', torch.cuda.is_available())
if torch.cuda.is_available():
   print('B16 supported: ', torch.cuda.is_bf16_supported())
   torch.cuda.set_device(0)
   print('Current device: ', torch.cuda.current_device())
   print('CUDA device name: ', torch.cuda.get_device_name(0))

## Fine tuning the LLM Model with Low-Rank Adaptation (LoRA) / Parameter Efficient Fine Tuning (PEFT)

We will add a LoRA adapter to the LLM (flan-t5-base) and train the adapter. The original LLM will be frozen. The adapter can be combined with the original LLM during inferencing. 

In [64]:
# TODO: Configure LoRA


In [65]:
# TODO: Add LoRA to the LLM model to be trained



In [ ]:
# TODO: Print number of parameters, compare LoRA to the original model



In [67]:
# TODO: Train model with LoRA
output_dir = f'peft-dialog-summary-training-{str(int(time.time()))}'



In [68]:
# TODO: Create trainer and train model
# TODO: Save trained model



### Use a trained LoRA model

The training will take a few hours and over many iterations.

For the purpose of this workshop we use a save model [intotheverse/peft-dialogue-summary-checkpoint](https://huggingface.co/intotheverse/peft-dialogue-summary-checkpoint).

In [ ]:
#TODO: Load the original model and add the pre-trained LoRA adaptation to the model
peft_dialogue_summary_checkpoint = 'intotheverse/peft-dialogue-summary-checkpoint'



## Evaluate LoRA model

In [ ]:
# Evaluate LoRA model with a single sample



In [ ]:
# TODO: Compare LoRA against the original model 

dialogues = []
summaries = []
orig_model_summaries = []
lora_model_summaries = []
config = GenerationConfig(max_new_tokens=200)



### Evaluate models with ROUGE metrics

Recall-Oriented Understudy for Gisting Evaluate ([ROUGE](https://pub.aimind.so/unveiling-the-power-of-rouge-metrics-in-nlp-b6d3f96d3363)) is a set of metrics used to evaluate the quality of machine-generated text, such as summaries and translations. ROUGE metrics compare the generated text to a human-written reference and measure the overlap between the two. 

The metrics range between 0 and 1, with higher scores indicating higher similarity between the baseline and generated text.

In [72]:
# TODO: create ROUGE


In [ ]:
# TODO: Evaluate the original model's result
